In [81]:
import os
import numpy as np
import pandas as pd
import random
from skimage import exposure

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model,load_model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping, ModelCheckpoint

In [91]:
spiral_train_path = '../input/parkinsons-drawings/spiral/training/'
wave_train_path = '../input/parkinsons-drawings/wave/training'
spiral_val_path = '../input/parkinsons-drawings/spiral/testing/'
wave_val_path = '../input/parkinsons-drawings/wave/testing'

In [83]:
init = tf.keras.initializers.glorot_normal()
reg = tf.keras.regularizers.L2(0.001)
batch_size = 24

In [85]:
def eqz_plz(img):
    return exposure.equalize_hist(img)

train_datagen = ImageDataGenerator(rotation_range=360, # they're spirals.
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    brightness_range=(0.5,1.5),
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    preprocessing_function = eqz_plz,
                                    vertical_flip=True)

train_gen = train_datagen.flow_from_directory(directory = os.path.abspath(spiral_train_path), target_size = (256,256), batch_size = batch_size,
                                              color_mode = 'grayscale', class_mode = 'binary', shuffle = True, seed = 101)

val_gen = train_datagen.flow_from_directory(directory = os.path.abspath(spiral_val_path), target_size = (256,256), batch_size = batch_size,
                                            color_mode = 'grayscale', class_mode = 'binary', shuffle = True, seed = 101)

Found 72 images belonging to 2 classes.
Found 30 images belonging to 2 classes.


In [86]:
def net(mode):
    if (mode == 'spirals') or (mode == 'spiral'):
        input_layer = Input(shape=(256,256,1),name=f'{mode}_input_layer')
    elif (mode == 'waves') or (mode == 'wave'):
        input_layer = Input(shape=(256,512,1),name=f'{mode}_input_layer')

    m1 = Conv2D(256,(5,5),dilation_rate=4,kernel_initializer=init,kernel_regularizer=reg,activation='relu',padding='same')(input_layer)
    p1 = MaxPool2D((9,9),strides=3)(m1)
    m2 = Conv2D(128,(5,5),dilation_rate=2,kernel_initializer=init,kernel_regularizer=reg,activation='relu',padding='same')(p1)
    p2 = MaxPool2D((7,7),strides=3)(m2)
    m3 = Conv2D(64,(3,3),kernel_initializer=init,kernel_regularizer=reg,activation='relu',padding='same')(p2)
    p3 = MaxPool2D((5,5),strides=2)(m3)
    f1 = Flatten()(p3)
    d1 = Dense(666,activation='relu')(f1)
    d2 = Dense(1,activation='sigmoid')(d1)
    
    this_model = Model(input_layer,d2)
    #this_model.summary()
    return this_model

In [87]:
spiral_model = net(mode='spirals') # early stopping epoch 89: val_loss 0.4796, val_acc 0.8274
spiral_model.compile(optimizer=Adam(lr=3.15e-5), loss='binary_crossentropy', metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss',patience=12,min_lr=1e-9,verbose=1)
early_stop = EarlyStopping(monitor='val_loss',patience=16,verbose=1)
checkpoint = ModelCheckpoint('best_weights.hdf5', save_best_only = True, save_weights_only = True, verbose = 1,
                             monitor = 'val_accuracy', mode = 'max')

In [88]:
spiral_model.fit_generator(train_gen,
                           validation_data = val_gen,
                           epochs=666,
#                            steps_per_epoch=(200//batch_size),
#                            validation_steps=(80//batch_size),
                           callbacks=[reduce_lr, checkpoint],
                           verbose=1)

/opt/conda/lib/python3.7/site-packages/skimage/exposure/exposure.py:181: UserWarning: This might be a color image. The histogram will be computed on the flattened image. You can instead apply this function to each color channel.
  hist, bin_centers = histogram(image, nbins)


Epoch 1/666
3/3 [==============================] - ETA: 0s - loss: 0.9689 - accuracy: 0.3611
Epoch 00001: val_accuracy improved from -inf to 0.50000, saving model to best_weights.hdf5
3/3 [==============================] - 2s 514ms/step - loss: 0.9689 - accuracy: 0.3611 - val_loss: 0.9501 - val_accuracy: 0.5000
Epoch 2/666
3/3 [==============================] - ETA: 0s - loss: 0.9508 - accuracy: 0.5278
Epoch 00002: val_accuracy did not improve from 0.50000
3/3 [==============================] - 1s 338ms/step - loss: 0.9508 - accuracy: 0.5278 - val_loss: 0.9499 - val_accuracy: 0.5000
Epoch 3/666
3/3 [==============================] - ETA: 0s - loss: 0.9507 - accuracy: 0.5000
Epoch 00003: val_accuracy did not improve from 0.50000
3/3 [==============================] - 1s 334ms/step - loss: 0.9507 - accuracy: 0.5000 - val_loss: 0.9552 - val_accuracy: 0.5000
Epoch 4/666
3/3 [==============================] - ETA: 0s - loss: 0.9492 - accuracy: 0.5000
Epoch 00004: val_accuracy did not improv

Epoch 30/666
3/3 [==============================] - ETA: 0s - loss: 0.9313 - accuracy: 0.4722
Epoch 00030: val_accuracy did not improve from 0.70000
3/3 [==============================] - 1s 341ms/step - loss: 0.9313 - accuracy: 0.4722 - val_loss: 0.9297 - val_accuracy: 0.5333
Epoch 31/666
3/3 [==============================] - ETA: 0s - loss: 0.9248 - accuracy: 0.5694
Epoch 00031: val_accuracy did not improve from 0.70000
3/3 [==============================] - 1s 333ms/step - loss: 0.9248 - accuracy: 0.5694 - val_loss: 0.9264 - val_accuracy: 0.5000
Epoch 32/666
3/3 [==============================] - ETA: 0s - loss: 0.9245 - accuracy: 0.5694
Epoch 00032: val_accuracy did not improve from 0.70000
3/3 [==============================] - 1s 332ms/step - loss: 0.9245 - accuracy: 0.5694 - val_loss: 0.9247 - val_accuracy: 0.5333
Epoch 33/666
3/3 [==============================] - ETA: 0s - loss: 0.9294 - accuracy: 0.4306
Epoch 00033: val_accuracy did not improve from 0.70000
3/3 [============

3/3 [==============================] - ETA: 0s - loss: 0.9073 - accuracy: 0.5556
Epoch 00059: val_accuracy did not improve from 0.73333
3/3 [==============================] - 1s 346ms/step - loss: 0.9073 - accuracy: 0.5556 - val_loss: 0.9109 - val_accuracy: 0.4667
Epoch 60/666
3/3 [==============================] - ETA: 0s - loss: 0.9054 - accuracy: 0.5556
Epoch 00060: val_accuracy did not improve from 0.73333
3/3 [==============================] - 1s 335ms/step - loss: 0.9054 - accuracy: 0.5556 - val_loss: 0.9044 - val_accuracy: 0.5333
Epoch 61/666
3/3 [==============================] - ETA: 0s - loss: 0.9106 - accuracy: 0.5139
Epoch 00061: val_accuracy did not improve from 0.73333
3/3 [==============================] - 1s 336ms/step - loss: 0.9106 - accuracy: 0.5139 - val_loss: 0.9004 - val_accuracy: 0.5333
Epoch 62/666
3/3 [==============================] - ETA: 0s - loss: 0.9085 - accuracy: 0.5278
Epoch 00062: val_accuracy did not improve from 0.73333
3/3 [=========================

Epoch 89/666
3/3 [==============================] - ETA: 0s - loss: 0.8875 - accuracy: 0.5000
Epoch 00089: val_accuracy did not improve from 0.73333
3/3 [==============================] - 1s 352ms/step - loss: 0.8875 - accuracy: 0.5000 - val_loss: 0.9025 - val_accuracy: 0.4333
Epoch 90/666
3/3 [==============================] - ETA: 0s - loss: 0.8864 - accuracy: 0.5278
Epoch 00090: val_accuracy improved from 0.73333 to 0.80000, saving model to best_weights.hdf5
3/3 [==============================] - 1s 371ms/step - loss: 0.8864 - accuracy: 0.5278 - val_loss: 0.8492 - val_accuracy: 0.8000
Epoch 91/666
3/3 [==============================] - ETA: 0s - loss: 0.8812 - accuracy: 0.5694
Epoch 00091: val_accuracy did not improve from 0.80000
3/3 [==============================] - 1s 342ms/step - loss: 0.8812 - accuracy: 0.5694 - val_loss: 0.8660 - val_accuracy: 0.6667
Epoch 92/666
3/3 [==============================] - ETA: 0s - loss: 0.8838 - accuracy: 0.5417
Epoch 00092: val_accuracy did not

...

Epoch 534/666
3/3 [==============================] - ETA: 0s - loss: 0.8700 - accuracy: 0.6944
Epoch 00534: val_accuracy did not improve from 0.90000
3/3 [==============================] - 1s 337ms/step - loss: 0.8700 - accuracy: 0.6944 - val_loss: 0.8808 - val_accuracy: 0.6000
Epoch 535/666
3/3 [==============================] - ETA: 0s - loss: 0.8742 - accuracy: 0.5972
Epoch 00535: val_accuracy did not improve from 0.90000
3/3 [==============================] - 1s 344ms/step - loss: 0.8742 - accuracy: 0.5972 - val_loss: 0.8807 - val_accuracy: 0.6333
Epoch 536/666
3/3 [==============================] - ETA: 0s - loss: 0.8804 - accuracy: 0.5972
Epoch 00536: val_accuracy did not improve from 0.90000
3/3 [==============================] - 1s 332ms/step - loss: 0.8804 - accuracy: 0.5972 - val_loss: 0.8752 - val_accuracy: 0.6333
Epoch 537/666
3/3 [==============================] - ETA: 0s - loss: 0.8781 - accuracy: 0.6111
Epoch 00537: val_accuracy did not improve from 0.90000
3/3 [========

Epoch 564/666
3/3 [==============================] - ETA: 0s - loss: 0.8763 - accuracy: 0.6111
Epoch 00564: val_accuracy did not improve from 0.90000
3/3 [==============================] - 1s 331ms/step - loss: 0.8763 - accuracy: 0.6111 - val_loss: 0.8783 - val_accuracy: 0.5333
Epoch 565/666
3/3 [==============================] - ETA: 0s - loss: 0.8847 - accuracy: 0.5556
Epoch 00565: val_accuracy did not improve from 0.90000
3/3 [==============================] - 1s 343ms/step - loss: 0.8847 - accuracy: 0.5556 - val_loss: 0.8835 - val_accuracy: 0.6000
Epoch 566/666
3/3 [==============================] - ETA: 0s - loss: 0.8747 - accuracy: 0.6389
Epoch 00566: val_accuracy did not improve from 0.90000
3/3 [==============================] - 1s 344ms/step - loss: 0.8747 - accuracy: 0.6389 - val_loss: 0.8820 - val_accuracy: 0.6000
Epoch 567/666
3/3 [==============================] - ETA: 0s - loss: 0.8795 - accuracy: 0.6111
Epoch 00567: val_accuracy did not improve from 0.90000
3/3 [========

Epoch 594/666
3/3 [==============================] - ETA: 0s - loss: 0.8782 - accuracy: 0.5833
Epoch 00594: val_accuracy did not improve from 0.90000
3/3 [==============================] - 1s 375ms/step - loss: 0.8782 - accuracy: 0.5833 - val_loss: 0.8806 - val_accuracy: 0.5333
Epoch 595/666
3/3 [==============================] - ETA: 0s - loss: 0.8846 - accuracy: 0.5694
Epoch 00595: val_accuracy did not improve from 0.90000
3/3 [==============================] - 1s 357ms/step - loss: 0.8846 - accuracy: 0.5694 - val_loss: 0.8805 - val_accuracy: 0.6333
Epoch 596/666
3/3 [==============================] - ETA: 0s - loss: 0.8748 - accuracy: 0.6528
Epoch 00596: val_accuracy did not improve from 0.90000
3/3 [==============================] - 1s 339ms/step - loss: 0.8748 - accuracy: 0.6528 - val_loss: 0.8757 - val_accuracy: 0.6333
Epoch 597/666
3/3 [==============================] - ETA: 0s - loss: 0.8735 - accuracy: 0.6528
Epoch 00597: val_accuracy did not improve from 0.90000
3/3 [========

Epoch 624/666
3/3 [==============================] - ETA: 0s - loss: 0.8780 - accuracy: 0.5972
Epoch 00624: val_accuracy did not improve from 0.90000
3/3 [==============================] - 1s 351ms/step - loss: 0.8780 - accuracy: 0.5972 - val_loss: 0.8776 - val_accuracy: 0.6333
Epoch 625/666
3/3 [==============================] - ETA: 0s - loss: 0.8756 - accuracy: 0.6389
Epoch 00625: val_accuracy did not improve from 0.90000
3/3 [==============================] - 1s 370ms/step - loss: 0.8756 - accuracy: 0.6389 - val_loss: 0.8785 - val_accuracy: 0.6333
Epoch 626/666
3/3 [==============================] - ETA: 0s - loss: 0.8772 - accuracy: 0.5833
Epoch 00626: val_accuracy did not improve from 0.90000
3/3 [==============================] - 1s 425ms/step - loss: 0.8772 - accuracy: 0.5833 - val_loss: 0.8746 - val_accuracy: 0.6667
Epoch 627/666
3/3 [==============================] - ETA: 0s - loss: 0.8804 - accuracy: 0.5972
Epoch 00627: val_accuracy did not improve from 0.90000
3/3 [========

Epoch 654/666
3/3 [==============================] - ETA: 0s - loss: 0.8728 - accuracy: 0.6667
Epoch 00654: val_accuracy did not improve from 0.90000
3/3 [==============================] - 1s 341ms/step - loss: 0.8728 - accuracy: 0.6667 - val_loss: 0.8812 - val_accuracy: 0.6000
Epoch 655/666
3/3 [==============================] - ETA: 0s - loss: 0.8765 - accuracy: 0.6667
Epoch 00655: val_accuracy did not improve from 0.90000
3/3 [==============================] - 1s 401ms/step - loss: 0.8765 - accuracy: 0.6667 - val_loss: 0.8854 - val_accuracy: 0.5000
Epoch 656/666
3/3 [==============================] - ETA: 0s - loss: 0.8840 - accuracy: 0.5833
Epoch 00656: val_accuracy did not improve from 0.90000
3/3 [==============================] - 1s 344ms/step - loss: 0.8840 - accuracy: 0.5833 - val_loss: 0.8757 - val_accuracy: 0.6333
Epoch 657/666
3/3 [==============================] - ETA: 0s - loss: 0.8689 - accuracy: 0.6806
Epoch 00657: val_accuracy did not improve from 0.90000
3/3 [========

In [90]:
spiral_model.save('../nopamine_model_spirals.h5')

In [92]:
wave_datagen = ImageDataGenerator(rotation_range=5,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  brightness_range=(0.5,1.5),
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  preprocessing_function=eqz_plz,
                                  vertical_flip=True)
wave_train_generator = wave_datagen.flow_from_directory(directory=os.path.abspath(wave_train_path),
                                                        target_size=(256, 512), # HxW in machine learning, WxH in computer vision
                                                        color_mode="grayscale",
                                                        batch_size=batch_size,
                                                        class_mode="binary",
                                                        shuffle=True,
                                                        seed=420)

wave_val_generator = wave_datagen.flow_from_directory(directory=os.path.abspath(wave_val_path),
                                                        target_size=(256, 512),
                                                        color_mode="grayscale",
                                                        batch_size=batch_size,
                                                        class_mode="binary",
                                                        shuffle=True,
                                                        seed=420)

Found 72 images belonging to 2 classes.
Found 30 images belonging to 2 classes.


In [96]:
waves_model = net(mode='waves')
waves_model.compile(optimizer=Adam(lr=3.15e-5),loss='binary_crossentropy',metrics=['accuracy'])
checkpoint_wave = ModelCheckpoint('best_weights_waves.hdf5', save_best_only = True, save_weights_only = True, verbose = 1,
                             monitor = 'val_accuracy', mode = 'max')

In [97]:
waves_model.fit_generator(wave_train_generator,
                          validation_data=wave_val_generator,
                          epochs=666,
#                           steps_per_epoch=(2000//batch_size),
#                           validation_steps=(800//batch_size),
                          callbacks=[reduce_lr,checkpoint_wave],
                          verbose=1)

Epoch 1/666
3/3 [==============================] - ETA: 0s - loss: 0.9876 - accuracy: 0.5000
Epoch 00001: val_accuracy improved from -inf to 0.50000, saving model to best_weights_waves.hdf5
3/3 [==============================] - 3s 866ms/step - loss: 0.9876 - accuracy: 0.5000 - val_loss: 0.9529 - val_accuracy: 0.5000
Epoch 2/666
3/3 [==============================] - ETA: 0s - loss: 0.9515 - accuracy: 0.4722
Epoch 00002: val_accuracy did not improve from 0.50000
3/3 [==============================] - 2s 633ms/step - loss: 0.9515 - accuracy: 0.4722 - val_loss: 0.9513 - val_accuracy: 0.5000
Epoch 3/666
3/3 [==============================] - ETA: 0s - loss: 0.9531 - accuracy: 0.5000
Epoch 00003: val_accuracy did not improve from 0.50000
3/3 [==============================] - 2s 783ms/step - loss: 0.9531 - accuracy: 0.5000 - val_loss: 0.9522 - val_accuracy: 0.5000
Epoch 4/666
3/3 [==============================] - ETA: 0s - loss: 0.9500 - accuracy: 0.4861
Epoch 00004: val_accuracy did not 

3/3 [==============================] - ETA: 0s - loss: 0.9130 - accuracy: 0.6111
Epoch 00030: val_accuracy did not improve from 0.73333
3/3 [==============================] - 2s 639ms/step - loss: 0.9130 - accuracy: 0.6111 - val_loss: 0.9032 - val_accuracy: 0.5667
Epoch 31/666
3/3 [==============================] - ETA: 0s - loss: 0.9218 - accuracy: 0.5000
Epoch 00031: val_accuracy did not improve from 0.73333
3/3 [==============================] - 2s 636ms/step - loss: 0.9218 - accuracy: 0.5000 - val_loss: 0.8988 - val_accuracy: 0.6667
Epoch 32/666
3/3 [==============================] - ETA: 0s - loss: 0.9064 - accuracy: 0.5833
Epoch 00032: val_accuracy did not improve from 0.73333
3/3 [==============================] - 2s 640ms/step - loss: 0.9064 - accuracy: 0.5833 - val_loss: 0.8979 - val_accuracy: 0.5000
Epoch 33/666
3/3 [==============================] - ETA: 0s - loss: 0.9096 - accuracy: 0.5139
Epoch 00033: val_accuracy did not improve from 0.73333
3/3 [=========================

3/3 [==============================] - ETA: 0s - loss: 0.8021 - accuracy: 0.7361
Epoch 00059: val_accuracy did not improve from 0.83333
3/3 [==============================] - 2s 615ms/step - loss: 0.8021 - accuracy: 0.7361 - val_loss: 0.7812 - val_accuracy: 0.7333
Epoch 60/666
3/3 [==============================] - ETA: 0s - loss: 0.8336 - accuracy: 0.6806
Epoch 00060: val_accuracy did not improve from 0.83333
3/3 [==============================] - 2s 649ms/step - loss: 0.8336 - accuracy: 0.6806 - val_loss: 0.8067 - val_accuracy: 0.7000
Epoch 61/666
3/3 [==============================] - ETA: 0s - loss: 0.7828 - accuracy: 0.7500
Epoch 00061: val_accuracy did not improve from 0.83333
3/3 [==============================] - 2s 629ms/step - loss: 0.7828 - accuracy: 0.7500 - val_loss: 0.7602 - val_accuracy: 0.7333
Epoch 62/666
3/3 [==============================] - ETA: 0s - loss: 0.8291 - accuracy: 0.6944
Epoch 00062: val_accuracy did not improve from 0.83333
3/3 [=========================

3/3 [==============================] - ETA: 0s - loss: 0.7477 - accuracy: 0.7083
Epoch 00088: val_accuracy did not improve from 0.90000
3/3 [==============================] - 2s 643ms/step - loss: 0.7477 - accuracy: 0.7083 - val_loss: 0.7601 - val_accuracy: 0.8000
Epoch 89/666
3/3 [==============================] - ETA: 0s - loss: 0.7464 - accuracy: 0.7639
Epoch 00089: val_accuracy did not improve from 0.90000
3/3 [==============================] - 2s 636ms/step - loss: 0.7464 - accuracy: 0.7639 - val_loss: 0.6476 - val_accuracy: 0.7667
Epoch 90/666
3/3 [==============================] - ETA: 0s - loss: 0.7432 - accuracy: 0.7222
Epoch 00090: val_accuracy did not improve from 0.90000
3/3 [==============================] - 2s 684ms/step - loss: 0.7432 - accuracy: 0.7222 - val_loss: 0.7564 - val_accuracy: 0.6667
Epoch 91/666
3/3 [==============================] - ETA: 0s - loss: 0.7569 - accuracy: 0.6806
Epoch 00091: val_accuracy did not improve from 0.90000
3/3 [=========================

3/3 [==============================] - ETA: 0s - loss: 0.7310 - accuracy: 0.7500
Epoch 00117: val_accuracy did not improve from 0.90000
3/3 [==============================] - 2s 624ms/step - loss: 0.7310 - accuracy: 0.7500 - val_loss: 0.6600 - val_accuracy: 0.8333
Epoch 118/666
3/3 [==============================] - ETA: 0s - loss: 0.7264 - accuracy: 0.7917
Epoch 00118: val_accuracy did not improve from 0.90000
3/3 [==============================] - 2s 669ms/step - loss: 0.7264 - accuracy: 0.7917 - val_loss: 0.6390 - val_accuracy: 0.9000
Epoch 119/666
3/3 [==============================] - ETA: 0s - loss: 0.7177 - accuracy: 0.7500
Epoch 00119: val_accuracy did not improve from 0.90000
3/3 [==============================] - 2s 619ms/step - loss: 0.7177 - accuracy: 0.7500 - val_loss: 0.7253 - val_accuracy: 0.7333
Epoch 120/666
3/3 [==============================] - ETA: 0s - loss: 0.7450 - accuracy: 0.7222
Epoch 00120: val_accuracy did not improve from 0.90000
3/3 [======================

...

Epoch 534/666
3/3 [==============================] - ETA: 0s - loss: 0.7360 - accuracy: 0.7917
Epoch 00534: val_accuracy did not improve from 0.96667
3/3 [==============================] - 2s 657ms/step - loss: 0.7360 - accuracy: 0.7917 - val_loss: 0.6948 - val_accuracy: 0.8000
Epoch 535/666
3/3 [==============================] - ETA: 0s - loss: 0.8078 - accuracy: 0.6389
Epoch 00535: val_accuracy did not improve from 0.96667
3/3 [==============================] - 2s 620ms/step - loss: 0.8078 - accuracy: 0.6389 - val_loss: 0.7434 - val_accuracy: 0.6667
Epoch 536/666
3/3 [==============================] - ETA: 0s - loss: 0.7395 - accuracy: 0.7361
Epoch 00536: val_accuracy did not improve from 0.96667
3/3 [==============================] - 2s 644ms/step - loss: 0.7395 - accuracy: 0.7361 - val_loss: 0.6654 - val_accuracy: 0.8000
Epoch 537/666
3/3 [==============================] - ETA: 0s - loss: 0.7012 - accuracy: 0.7222
Epoch 00537: val_accuracy did not improve from 0.96667
3/3 [========

Epoch 564/666
3/3 [==============================] - ETA: 0s - loss: 0.7983 - accuracy: 0.6806
Epoch 00564: val_accuracy did not improve from 0.96667
3/3 [==============================] - 2s 610ms/step - loss: 0.7983 - accuracy: 0.6806 - val_loss: 0.7583 - val_accuracy: 0.8333
Epoch 565/666
3/3 [==============================] - ETA: 0s - loss: 0.7790 - accuracy: 0.7361
Epoch 00565: val_accuracy did not improve from 0.96667
3/3 [==============================] - 2s 621ms/step - loss: 0.7790 - accuracy: 0.7361 - val_loss: 0.6124 - val_accuracy: 0.8000
Epoch 566/666
3/3 [==============================] - ETA: 0s - loss: 0.7558 - accuracy: 0.7500
Epoch 00566: val_accuracy did not improve from 0.96667
3/3 [==============================] - 2s 626ms/step - loss: 0.7558 - accuracy: 0.7500 - val_loss: 0.6830 - val_accuracy: 0.7667
Epoch 567/666
3/3 [==============================] - ETA: 0s - loss: 0.7753 - accuracy: 0.7222
Epoch 00567: val_accuracy did not improve from 0.96667
3/3 [========

Epoch 594/666
3/3 [==============================] - ETA: 0s - loss: 0.7378 - accuracy: 0.7361
Epoch 00594: val_accuracy did not improve from 0.96667
3/3 [==============================] - 2s 663ms/step - loss: 0.7378 - accuracy: 0.7361 - val_loss: 0.6994 - val_accuracy: 0.8333
Epoch 595/666
3/3 [==============================] - ETA: 0s - loss: 0.7807 - accuracy: 0.6944
Epoch 00595: val_accuracy did not improve from 0.96667
3/3 [==============================] - 2s 623ms/step - loss: 0.7807 - accuracy: 0.6944 - val_loss: 0.7074 - val_accuracy: 0.7333
Epoch 596/666
3/3 [==============================] - ETA: 0s - loss: 0.7787 - accuracy: 0.7083
Epoch 00596: val_accuracy did not improve from 0.96667
3/3 [==============================] - 2s 620ms/step - loss: 0.7787 - accuracy: 0.7083 - val_loss: 0.6640 - val_accuracy: 0.7667
Epoch 597/666
3/3 [==============================] - ETA: 0s - loss: 0.7164 - accuracy: 0.7639
Epoch 00597: val_accuracy did not improve from 0.96667
3/3 [========

Epoch 624/666
3/3 [==============================] - ETA: 0s - loss: 0.7294 - accuracy: 0.7917
Epoch 00624: val_accuracy did not improve from 0.96667
3/3 [==============================] - 2s 666ms/step - loss: 0.7294 - accuracy: 0.7917 - val_loss: 0.7502 - val_accuracy: 0.7333
Epoch 625/666
3/3 [==============================] - ETA: 0s - loss: 0.7707 - accuracy: 0.6667
Epoch 00625: val_accuracy did not improve from 0.96667
3/3 [==============================] - 2s 628ms/step - loss: 0.7707 - accuracy: 0.6667 - val_loss: 0.7800 - val_accuracy: 0.7667
Epoch 626/666
3/3 [==============================] - ETA: 0s - loss: 0.7949 - accuracy: 0.7083
Epoch 00626: val_accuracy did not improve from 0.96667
3/3 [==============================] - 2s 608ms/step - loss: 0.7949 - accuracy: 0.7083 - val_loss: 0.6721 - val_accuracy: 0.7333
Epoch 627/666
3/3 [==============================] - ETA: 0s - loss: 0.7132 - accuracy: 0.7778
Epoch 00627: val_accuracy did not improve from 0.96667
3/3 [========

Epoch 654/666
3/3 [==============================] - ETA: 0s - loss: 0.7397 - accuracy: 0.7083
Epoch 00654: val_accuracy did not improve from 0.96667
3/3 [==============================] - 2s 625ms/step - loss: 0.7397 - accuracy: 0.7083 - val_loss: 0.6959 - val_accuracy: 0.8333
Epoch 655/666
3/3 [==============================] - ETA: 0s - loss: 0.6923 - accuracy: 0.7639
Epoch 00655: val_accuracy did not improve from 0.96667
3/3 [==============================] - 2s 618ms/step - loss: 0.6923 - accuracy: 0.7639 - val_loss: 0.6999 - val_accuracy: 0.8000
Epoch 656/666
3/3 [==============================] - ETA: 0s - loss: 0.7407 - accuracy: 0.7083
Epoch 00656: val_accuracy did not improve from 0.96667
3/3 [==============================] - 2s 686ms/step - loss: 0.7407 - accuracy: 0.7083 - val_loss: 0.6919 - val_accuracy: 0.7667
Epoch 657/666
3/3 [==============================] - ETA: 0s - loss: 0.7228 - accuracy: 0.7222
Epoch 00657: val_accuracy did not improve from 0.96667
3/3 [========